https://www.1point3acres.com/bbs/thread-1086943-1-1.html

### Part 1
题干如下：
subscription: list of users {name, plan, begin_date, duration}，要求按顺序发email (plan当天发welcome, -15 days发upcoming expiration，expire当天发expire)

例如：
```python
users = [{name: A, plan: X, begin_date = 0, duration = 30}, {name: B, plan: Y, begin_date = 1, duration = 15}]
```
输出
```
0: [Welcome] A, subscribe in plan X
1: [Welcome] B, subscribe in plan Y
1: [Upcoming expiration] B, subscribe in plan Y
15: [Upcoming expiration] A, subscribe in plan X
16: [Expired] B, subcribe in plan Y
30: [Expired] A, subscribe in plan X
```
### Part 2
第二问：基础上增加一个change in plans, list of changes {name, new_plan, change_date}

```python
users = [{name: A, plan: X, begin_date = 0, duration = 30}, {name: B, plan: Y, begin_date = 1, duration = 15}]
changes = [{name: A, new_plan: Y, change_date = 5}]
输出
0: [Welcome] A, subscribe in plan X
1: [Welcome] B, subscribe in plan Y
1: [Upcoming expiration] B, subscribe in plan Y
5: [Changed] A, subscribe in plan Y
15: [Upcoming expiration] A, subscribe in plan Y
16: [Expired] B, subcribe in plan Y
30: [Expired] A, subscribe in plan Y```

第三问是bonus，第二问的基础上再加一个renew，change list中entry为 {name, extension, change_date}


In [78]:
def email_subscription_pt1(users):
    email_list = []

    for user in users:
        # add welcome email
        welcome_email = f"{user['begin_date']}: [Welcome] {user['name']}, subscribe in plan {user['plan']}"
        email_list.append(welcome_email)

        # add upcoming expiration email
        reminder_email = f"{user['begin_date'] + user['duration'] - 15}: [Upcoming expiration] {user['name']}, subscribe in plan {user['plan']}"
        email_list.append(reminder_email)

        # add expiration email
        exp_email = f"{user['begin_date'] + user['duration']}: [Expired] {user['name']}, subscribe in plan {user['plan']}"
        email_list.append(exp_email)

    return sorted(email_list, key=lambda x: int(x.split(':')[0]))

In [79]:
users = [
    {'name': 'A', 'plan': 'X', 'begin_date': 0, 'duration': 30}, 
    {'name': 'B', 'plan': 'Y', 'begin_date': 1, 'duration': 15}
]
email_subscription_pt1(users)

['0: [Welcome] A, subscribe in plan X',
 '1: [Welcome] B, subscribe in plan Y',
 '1: [Upcoming expiration] B, subscribe in plan Y',
 '15: [Upcoming expiration] A, subscribe in plan X',
 '16: [Expired] B, subscribe in plan Y',
 '30: [Expired] A, subscribe in plan X']

In [83]:
def email_subscription_pt2(users, changes):
    email_list = []

    users_with_changes = [change['name'] for change in changes]

    for record in users:
        user, plan, begin_date, reminder_date, end_date = record['name'], record['plan'], int(record['begin_date']), int(record['begin_date']) + int(record['duration']) - 15, int(record['begin_date']) + int(record['duration'])
        if user not in users_with_changes:
            email_list.append(f"{begin_date}: [Welcome] {user}, subscribe in plan {plan}")
            email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
            email_list.append(f"{end_date}: [Expired] {user}, subscribe in plan {plan}")
        else:    
            for change in changes:
                # check if there are associated changes for this user
                if change['name'] == user:
                    email_list.append(f"{begin_date}: [Welcome] {user}, subscribe in plan {plan}")
                    email_list.append(f"{change['change_date']}: [Changed] {user}, subscribe in plan {change['new_plan']}")
                    if change['change_date'] < reminder_date:
                        email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {change['new_plan']}")
                    else:
                        email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
                    email_list.append(f"{end_date}: [Expired] {user}, subscribe in plan {change['new_plan']}")
               
    return sorted(email_list, key=lambda x: int(x.split(':')[0]))

In [84]:
users = [
    {'name': 'A', 'plan': 'X', 'begin_date': 0, 'duration': 30}, 
    {'name': 'B', 'plan': 'Y', 'begin_date': 1, 'duration': 15}
]

changes = [{'name': 'A', 'new_plan': 'Y', 'change_date': 5}]

email_subscription_pt2(users, changes)

['0: [Welcome] A, subscribe in plan X',
 '1: [Welcome] B, subscribe in plan Y',
 '1: [Upcoming Expiration] B, subscribe in plan Y',
 '5: [Changed] A, subscribe in plan Y',
 '15: [Upcoming Expiration] A, subscribe in plan Y',
 '16: [Expired] B, subscribe in plan Y',
 '30: [Expired] A, subscribe in plan Y']

In [124]:
def email_subscription_pt3(users, changes):
    email_list = []

    users_with_changes = [change['name'] for change in changes]

    for record in users:
        user, plan, begin_date, reminder_date, end_date = record['name'], record['plan'], int(record['begin_date']), int(record['begin_date']) + int(record['duration']) - 15, int(record['begin_date']) + int(record['duration'])

        if user not in users_with_changes:
            email_list.append(f"{begin_date}: [Welcome] {user}, subscribe in plan {plan}")
            email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
            email_list.append(f"{end_date}: [Expired] {user}, subscribe in plan {plan}")            

        for change in changes:
            # check if there are associated changes for this user
            if change['name'] == user:
                # first situation of change
                if 'new_plan' in change:
                    email_list.append(f"{begin_date}: [Welcome] {user}, subscribe in plan {plan}")
                    email_list.append(f"{change['change_date']}: [Changed] {user}, subscribe in plan {change['new_plan']}")
                    if change['change_date'] < reminder_date:
                        email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {change['new_plan']}")
                    else:
                        email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
                    email_list.append(f"{end_date}: [Expired] {user}, subscribe in plan {change['new_plan']}")
                
                # second situation of change
                if 'extension' in change:
                    email_list.append(f"{begin_date}: [Welcome] {user}, subscribe in plan {plan}")
                    email_list.append(f"{change['change_date']}: [Changed] {user}, subscribe in plan {plan}")
                    if reminder_date < change['change_date']:
                        email_list.append(f"{reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
                    # calculate the new reminder date based on extension
                    new_end_date = end_date + change['extension']
                    new_reminder_date = reminder_date + change['extension']
                    email_list.append(f"{new_reminder_date}: [Upcoming Expiration] {user}, subscribe in plan {plan}")
                    email_list.append(f"{new_end_date}: [Expired] {user}, subscribe in plan {plan}")
        
    return sorted(email_list, key=lambda x: int(x.split(':')[0]))


In [125]:
users = [
    {'name': 'A', 'plan': 'X', 'begin_date': 0, 'duration': 30}, 
    {'name': 'B', 'plan': 'Y', 'begin_date': 1, 'duration': 15}
]

changes = [{'name': 'A', 'new_plan': 'Y', 'change_date': 5}, 
           {'name': 'B', 'extension': 15, 'change_date': 3}]

email_subscription_pt3(users, changes)


['0: [Welcome] A, subscribe in plan X',
 '1: [Welcome] B, subscribe in plan Y',
 '1: [Upcoming Expiration] B, subscribe in plan Y',
 '3: [Changed] B, subscribe in plan Y',
 '5: [Changed] A, subscribe in plan Y',
 '15: [Upcoming Expiration] A, subscribe in plan Y',
 '16: [Upcoming Expiration] B, subscribe in plan Y',
 '30: [Expired] A, subscribe in plan Y',
 '31: [Expired] B, subscribe in plan Y']

In [122]:
def email_subscription_pt3(users, changes):

    for record in users:
        print(record['name'])
        begin_date = record['begin_date']
        plans = [record['plan']]*record['duration']
        print(plans)

        for change in changes:
            if change['name'] == record['name']:
                # first situation of change: change plan
                if 'new_plan' in change:
                    start_of_change = change['change_date'] - begin_date 
                    plans[(start_of_change):] = change['new_plan'] * (len(plans) - start_of_change)
            
                if 'extension' in change:
                    new_plans = [record['plan']] * change['extension']
                    plans += new_plans
            
        print(plans)


In [123]:
email_subscription_pt3(users, changes)

A
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
B
['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
